In [1]:
import statistics
import queue
from collections import namedtuple
import numpy as np
import random
from tqdm import tqdm
from copy import deepcopy

from collections import Counter


In [10]:
with open('./data/gc_250_9', 'r') as input_data_file:
    input_data = input_data_file.read()
    
lines = input_data.split('\n')

first_line = lines[0].split()
node_count = int(first_line[0])
edge_count = int(first_line[1])

edges = []
for i in range(1, edge_count + 1):
    line = lines[i]
    parts = line.split()
    edges.append((int(parts[0]), int(parts[1])))

In [11]:
node_count, edge_count

(250, 28046)

In [12]:
class TabuList:
    def __init__(self, tabu_size):
        self.tabu_size = tabu_size
        self.tabu_hash = set()
        self.tabu_queue = []

    def is_present(self, node):
        return node in self.tabu_hash

    def insert(self, node):
        if self.is_present(node):
            return
        self.tabu_hash.add(node)
        self.tabu_queue.append(node)
        if len(self.tabu_hash) > self.tabu_size:
            self.remove()
            
    def remove(self):
        top = self.tabu_queue.pop(0)
        self.tabu_hash.remove(top)

In [13]:
def get_suboptimal_coloring(mapping):

    mapping = deepcopy(mapping)
    
    node_count = len(mapping)
    coloring = [[] for _ in range(node_count)]
    
    for idx in range(node_count):
        coloring[idx] = list(range(node_count))
    
    for start_node in range(node_count):
        
        if isinstance(coloring[start_node], list):
            coloring[start_node] = coloring[start_node][0]
        else:
            continue
            
        for j in mapping[start_node]:
                
            if coloring[start_node] in coloring[j]:
                coloring[j].remove(coloring[start_node])
                
            if start_node in mapping[j]:
                mapping[j].remove(start_node)

    del mapping
    return coloring

In [14]:
def get_conflicts(graph, coloring): 

    node_count = len(graph)
    conflicts = [0 for _ in range(node_count)]
    
    for node, neighbors in enumerate(graph):
        for neighbor in neighbors:
            if coloring[node] == coloring[neighbor]:
                conflicts[node] += 1
                
    return conflicts
    
def max_violation_node(conflict_mapping, tabu_list):
    
    max_conflicts = max([conflict for node, conflict in enumerate(conflict_mapping) if not tabu_list.is_present(node)])
    
    if max_conflicts == 0:
        return -1
        
    # Create a new dictionary with only the key-value pairs that have the maximum value and not in tabu list
    max_conflict_nodes = [node for node, conflict in enumerate(conflict_mapping) if conflict == max_conflicts and not tabu_list.is_present(node)]

    if not max_conflict_nodes:
        return -1
            
    return random.choice(max_conflict_nodes)


def remove_color(coloring, max_colors):
    
    pick_color = random.randint(0, max_colors-1)
    
    new_coloring = []
    
    for c in coloring:
        if c == pick_color:
            new_coloring.append(random.randint(0, max_colors-2))
        elif c > pick_color:
            new_coloring.append(c-1)
        else:
            new_coloring.append(c)
        
    return new_coloring
    

def unfamous_color(node, graph, coloring, max_colors):

    node_color = coloring[node]
    neighbor_colors = [coloring[color] for color in graph[node]]
    color_count = Counter(neighbor_colors)
    
    min_color = min([v for k, v in color_count.items() if k != node_color])
    
    unused_colors = list(set(range(max_colors))-set(color_count.keys()))
    
    if unused_colors:
        return random.choice(unused_colors)
    else:
        least_colors = [k for k, v in color_count.items() if v == min_color and k != node_color]
        return random.choice(least_colors)

In [15]:
def check_feasibility(graph, coloring, max_colors, tabu_limit):
    max_iterations = 50000
    iteration = 0
    conflicts = get_conflicts(graph, coloring)
    total_conflicts = sum(conflicts)
    tabu_list = TabuList(tabu_limit)

    # with tqdm(total=max_iterations) as pbar:
    while iteration < max_iterations and total_conflicts > 0:

        node = max_violation_node(conflicts, tabu_list)
    
        while node == -1:
            tabu_list.remove()
            node = max_violation_node(conflicts, tabu_list)
          
        tabu_list.insert(node)
        least_used_color = unfamous_color(node, graph, coloring, max_colors)

        for neighbor in graph[node]:
            if coloring[neighbor] == coloring[node]:
                conflicts[neighbor] -= 1
                conflicts[node] -= 1
            elif coloring[neighbor] == least_used_color:
                conflicts[neighbor] += 1
                conflicts[node] += 1
        
        coloring[node] = least_used_color
        
        # conflicts = get_conflicts(graph, coloring)
        total_conflicts = sum(conflicts)
        
        iteration += 1
            # pbar.update(1)

    return sum(conflicts) == 0, coloring, conflicts, iteration

In [16]:
def run_logic(connections, node_count):
    
    tabu_limit = node_count // 10
    
    mapping = [[] for _ in range(node_count)]
    
    for i, j in connections:
        mapping[i].append(j)
        mapping[j].append(i)

    new_coloring = get_suboptimal_coloring(mapping) #[random.randint(0, max_color-1) for _ in range(node_count)]
    max_color = len(set(new_coloring))

    # new_coloring = [random.randint(0, max_color-1) for _ in range(node_count)]
    # max_color = node_count
    
    max_retries = 20
    retry = max_retries

    
    while retry > 0:

        is_feasible, coloring, conflicts, iteration = check_feasibility(mapping, new_coloring, max_color, tabu_limit)
        print(max_color, is_feasible, len(set(coloring)), sum(conflicts), iteration)
        
        
        if is_feasible:
            max_color -= 1
            optimal_coloring = deepcopy(coloring)
            retry = max_retries

        else:
            retry -= 1


        new_coloring = remove_color(optimal_coloring, max_color)

    return optimal_coloring, coloring

In [17]:
optimal_coloring, last_check_coloring = run_logic(edges, node_count)

100 True 100 0 0
99 True 99 0 9
98 True 98 0 5
97 True 97 0 7
96 True 96 0 3
95 True 95 0 17
94 True 94 0 9
93 True 93 0 9
92 True 92 0 22
91 True 91 0 14
90 True 90 0 19
89 True 89 0 32
88 True 88 0 24
87 True 87 0 48
86 True 86 0 24
85 True 85 0 43
84 True 84 0 44
83 True 83 0 222
82 True 82 0 281
81 True 81 0 863
80 True 80 0 725
79 True 79 0 811
78 True 78 0 2386
77 True 77 0 10855
76 True 76 0 7431
75 False 75 38 50000
75 False 75 34 50000
75 False 75 32 50000
75 False 75 44 50000
75 False 75 32 50000
75 False 75 36 50000
75 False 75 40 50000
75 False 75 34 50000
75 False 75 26 50000
75 False 75 50 50000
75 False 75 28 50000
75 False 75 48 50000
75 False 75 34 50000
75 False 75 38 50000
75 False 75 24 50000
75 False 75 32 50000
75 False 75 26 50000
75 False 75 24 50000
75 False 75 22 50000
75 False 75 32 50000


In [ ]:
random.choice(list({1, 2, 3}))

In [ ]:
print(node, graph, coloring, max_colors)

In [ ]:
node = 1
mapping = [[3, 5, 8, 10, 42, 43, 45, 49], [5, 13, 15, 16, 20, 22, 24, 25, 27, 31, 35, 39, 45, 46, 47], [3, 9, 10, 12, 15, 21, 28, 36, 42, 46, 47, 48], [0, 2, 5, 7, 8, 9, 11, 25, 30, 33, 35, 36, 40, 45], [12, 15, 16, 20, 24, 26, 27, 30, 40, 43, 44, 45, 47], [0, 1, 3, 7, 18, 25, 32, 37, 42, 43, 45, 46, 49], [15, 16, 17, 18, 20, 23, 24, 28, 29, 33, 35, 37, 41, 43, 45, 46], [3, 5, 9, 14, 18, 20, 21, 22, 24, 26, 31, 32, 35, 37, 40, 41, 44, 46, 47, 49], [0, 3, 11, 16, 17, 20, 27, 28, 31, 32, 34, 41, 43, 47], [2, 3, 7, 10, 12, 14, 15, 19, 25, 26, 33, 35, 38, 43, 44], [0, 2, 9, 17, 19, 20, 30, 31, 37, 38, 39, 42], [3, 8, 15, 18, 20, 22, 28, 31, 32, 33, 34, 35, 37, 41, 46, 47], [2, 4, 9, 14, 16, 17, 20, 28, 29, 31, 43, 45, 48, 49], [1, 14, 15, 19, 20, 23, 24, 26, 29, 41], [7, 9, 12, 13, 19, 23, 26, 29, 37, 39, 48, 49], [1, 2, 4, 6, 9, 11, 13, 16, 19, 27, 30, 31, 33, 44, 48, 49], [1, 4, 6, 8, 12, 15, 17, 19, 25, 29, 30, 33, 35, 38, 41, 46, 48], [6, 8, 10, 12, 16, 22, 23, 29, 32, 33, 44, 45, 46], [5, 6, 7, 11, 22, 28, 31, 33, 35, 42], [9, 10, 13, 14, 15, 16, 23, 26, 29, 38, 39, 42, 43, 45, 47, 49], [1, 4, 6, 7, 8, 10, 11, 12, 13, 24, 28, 37, 40, 42, 44, 47], [2, 7, 22, 24, 25, 37, 44, 45], [1, 7, 11, 17, 18, 21, 27, 29, 30, 32, 41, 44, 47, 48, 49], [6, 13, 14, 17, 19, 27, 29, 32, 34, 42, 43], [1, 4, 6, 7, 13, 20, 21, 25, 27, 32, 40, 48], [1, 3, 5, 9, 16, 21, 24, 34, 45, 46, 48], [4, 7, 9, 13, 14, 19, 27, 29, 30, 32, 33, 36, 42, 48], [1, 4, 8, 15, 22, 23, 24, 26, 29, 30, 32, 35, 41, 44, 45, 47, 48], [2, 6, 8, 11, 12, 18, 20, 33, 34, 35, 36, 37, 43, 45], [6, 12, 13, 14, 16, 17, 19, 22, 23, 26, 27, 31, 35, 43, 44, 45], [3, 4, 10, 15, 16, 22, 26, 27, 31, 37, 39], [1, 7, 8, 10, 11, 12, 15, 18, 29, 30, 33, 36, 38, 40, 46, 48], [5, 7, 8, 11, 17, 22, 23, 24, 26, 27, 34, 35, 36, 39, 42, 45, 47, 49], [3, 6, 9, 11, 15, 16, 17, 18, 26, 28, 31, 34, 39, 41, 43, 44, 45, 46, 47], [8, 11, 23, 25, 28, 32, 33, 35, 40, 43, 44, 45], [1, 3, 6, 7, 9, 11, 16, 18, 27, 28, 29, 32, 34, 36, 39, 40, 41, 42, 45], [2, 3, 26, 28, 31, 32, 35, 42, 45, 47, 49], [5, 6, 7, 10, 11, 14, 20, 21, 28, 30, 38, 39, 43], [9, 10, 16, 19, 31, 37, 41, 42, 48], [1, 10, 14, 19, 30, 32, 33, 35, 37], [3, 4, 7, 20, 24, 31, 34, 35, 45, 46, 47], [6, 7, 8, 11, 13, 16, 22, 27, 33, 35, 38, 42, 44], [0, 2, 5, 10, 18, 19, 20, 23, 26, 32, 35, 36, 38, 41, 44, 47, 49], [0, 4, 5, 6, 8, 9, 12, 19, 23, 28, 29, 33, 34, 37, 44, 47], [4, 7, 9, 15, 17, 20, 21, 22, 27, 29, 33, 34, 41, 42, 43, 45, 49], [0, 1, 3, 4, 5, 6, 12, 17, 19, 21, 25, 27, 28, 29, 32, 33, 34, 35, 36, 40, 44, 47, 49], [1, 2, 5, 6, 7, 11, 16, 17, 25, 31, 33, 40, 47], [1, 2, 4, 7, 8, 11, 19, 20, 22, 27, 32, 33, 36, 40, 42, 43, 45, 46], [2, 12, 14, 15, 16, 22, 24, 25, 26, 27, 31, 38], [0, 5, 7, 12, 14, 15, 19, 22, 32, 36, 42, 44, 45]]
coloring = [1, 1, 3, 0, 1, 1, 1, 5, 1, 1, 0, 0, 0, 0, 2, 2, 3, 2, 0, 1, 2, 0, 1, 1, 1, 2, 3, 0, 3, 4, 4, 3, 4, 4, 0, 2, 1, 5, 2, 1, 4, 5, 6, 2, 3, 5, 0, 3, 4, 7]
max_colors = 8

In [ ]:
unfamous_color_test(node, mapping, coloring, max_colors)

In [ ]:
unfamous_color(node, mapping, coloring, max_colors)

In [ ]:
neighbor_colors = [coloring[color] for color in mapping[node]]
color_count = Counter(neighbor_colors)
color_count

In [ ]:
min_color = min([v for k, v in color_count.items() if k != coloring[node]])

In [ ]:
set(range(max_colors))-set(color_count.keys())

In [ ]:
neighbor_colors = [coloring[color] for color in mapping[node]]
color_count = Counter(neighbor_colors)

min_color = min([v for k, v in color_count.items() if k not in coloring[node]])

unused_colors = set(range(max_colors))-set(color_count.keys())

if unused_colors:
    return unused_colors
    # return random.choice(unused_colors)
else:
    least_colors = [k for k, v in color_count.items() if v == min_color and k != coloring[node]]
    return least_colors

In [ ]:
neighbor_colors = [coloring[color] for color in graph[node]]
    color_count = Counter(neighbor_colors)
    
    ## remove reusing same color as current node
    if coloring[node] in color_count:
        color_count.pop(coloring[node])
    
    min_color = min(color_count.values())
    
    unused_colors = set(range(max_colors))-set(color_count.keys())
    
    if unused_colors:
        return unused_colors
        # return random.choice(unused_colors)
    else:
        least_colors = [k for k, v in color_count.items() if v == min_color]
        return least_colors

In [ ]:
tabu_limit = node_count // 10
tabu_list = TabuList(tabu_limit)    
mapping = [[] for _ in range(node_count)]

for i, j in edges:
    mapping[i].append(j)
    mapping[j].append(i)

new_coloring = optimal_coloring #get_suboptimal_coloring(mapping) #[random.randint(0, max_color-1) for _ in range(node_count)]
max_color = len(set(new_coloring))

print(max_color)

In [ ]:
max_color = 6
new_coloring = remove_color(optimal_coloring, max_color)

In [ ]:
conflicts = get_conflicts(mapping, new_coloring)
sum(conflicts)

In [ ]:
max_violation_node(conflicts, tabu_list), max_violation_node_test(conflicts, tabu_list)

In [ ]:
tabu_list.tabu_queue

In [ ]:
node = 41
tabu_list.insert(node)

In [ ]:
unfamous_color(node, mapping, new_coloring, max_color), unfamous_color_test(node, mapping, new_coloring, max_color)

In [ ]:
least_used_color =4


new_coloring[node] = least_used_color

conflicts = get_conflicts(mapping, new_coloring)
sum(conflicts)